# Inserção de elementos
Notebook dedicado a incrementar as funções que serão usadas para adicionar

#### Importando bibliotecas e importando as funções primárias

In [1]:
#importa as bibliotecas
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
from unicodedata import normalize #biblioteca usada para criar as arvores TRIE

In [2]:
#Funcao que retira as features de artistas selecionados e salva apenas o artista principal da musica
def retiraFeaturing(artistaString):
    
    if " Featuring" in artistaString:   # Se a música tiver um featuring, deve-se retirá-lo
            indexFeat = artistaString.find(" Featuring")   # pega índice do char space antes de Featuring
            artistaString = artistaString[:indexFeat]   # retorna apenas o primeiro artista
        
    return artistaString   # caso não caia no if, simplesmente retorna o artista assim como veio

# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

#------------------------------------------------------------
#Definicao das funcoes usadas para gerar as arvores TRIE
#------------------------------------------------------------
#Definicao da classe de nodo das arvores TRIE
class TrieNode(object):
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista

#Definicao da funcao que adiciona uma string na arvore TRIE
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie 
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado
    
#Funcao que procura uma string na arvore TRIE 
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
#Funcoes utilizadas para sugerir buscas
def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:
            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 
            if filho.pFinalizada:   
                listaDeSugestoes.append(filho.indices[0])   # adiciona nova sugestão na lista
                
        return  
  
# Função auxiliar para adicionar artista levando-se em consideração a retirada do possível "feat" da string
# A função checa se a string passada como parâmetro contém uma sinalização de featuring para evitar que se crie um "artista novo"
#
'''def adicionaArtista(raiz, artista, indice):
    
    if " Featuring" in artista:   # Se a música tiver um featuring, deve-se retirá-lo
        indexFeat = artista.find(" Featuring")
        counter = 0
        artistaNewString = ""
        
        # como artista = [:indexFeat] dá erro de sintaxe, precisa-se iterar e construir outra string 
        # um loop for com contador (e break) é necessário nesse caso para poder iterar pela sting 
        for char in artista:
            if counter == indexFeat:
                break
            artistaNewString = artistaNewString + char
            counter += 1

        artista = artistaNewString   # atualiza nome do artista     

    addString(raiz, artista, indice)
'''

'def adicionaArtista(raiz, artista, indice):\n    \n    if " Featuring" in artista:   # Se a música tiver um featuring, deve-se retirá-lo\n        indexFeat = artista.find(" Featuring")\n        counter = 0\n        artistaNewString = ""\n        \n        # como artista = [:indexFeat] dá erro de sintaxe, precisa-se iterar e construir outra string \n        # um loop for com contador (e break) é necessário nesse caso para poder iterar pela sting \n        for char in artista:\n            if counter == indexFeat:\n                break\n            artistaNewString = artistaNewString + char\n            counter += 1\n\n        artista = artistaNewString   # atualiza nome do artista     \n\n    addString(raiz, artista, indice)\n'

In [216]:
#Aceita entradas no formato pandas.DataFrame no final do arquivo tem um exemplo de como criar essas estruturas a partir de dicionarios
#Implementado desse jeito pois nossos exemplos de inserção serão baseados em novas entradas do excel geradas anualmente pela Billboard
def adicionaAno(df):
    
    #----------------------------------------------------------
    #Abrindo os arquivos com os tops de artistas
    with (open('topArtistas.bin', 'rb')) as openfile:
        topArtistas = pickle.load(openfile)
        
    with (open('topMusicas.bin', 'rb')) as openfile:
        topMusicas = pickle.load(openfile)
        
    #Abrindo os arquivos com as arvores Trie      
    with (open('trieMusicas.bin', 'rb')) as openfile:
        trieMusicas = pickle.load(openfile)
        
    with (open('trieArtistas.bin', 'rb')) as openfile:
        trieArtistas = pickle.load(openfile)
    #----------------------------------------------------------
    #Salva os indices finais dos arquivos pré-inserção + 1 
    indiceIniArtist = len(topArtistas) 
    indiceIniMusic = len(topMusicas)
    contador = {}
    #Salva os registros do ano atual
    for i in range(len(df)):
        ano = df['WeekID'][0].split('-')
        ano = ano[0]
        if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
            contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
        else: #cai aqui se ainda nao foi computada
            contador[df['SongID'][i]] = {}
            contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
            contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
            contador[df['SongID'][i]]['Ano'] = ano
            contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
            #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
            contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
            contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
        
    #Faz o sort do ano atual
    items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
    #armazena as primeiras 200 entradas em um dicionário mais organizado
    top = [dict() for x in range(200)] #cria uma lista de 200 dicionarios
    for i in range(len(top)):
    
        for y in items[i][1]:
            top[i][y] = items[i][1][y]
        
        # Adiciona no dicionário de todos os artistas (ou soma os pontos) para cianção do arquivo topArtistas.bin
        artistaIterado = retiraFeaturing(str(top[i]['Artista']))   # retira o featuring caso tenha
        
        indArtista = -1 #flag que salva o indice do artista iterado, se for -1 o artista nao esta na lista
        indice = 0 #indice que esta procurando o artista
        for x in topArtistas: #percorre o arquivo topArtistas procurando pelo artista
            if x[0] == artistaIterado:
                indArtista = indice
            indice += 1

        if(indArtista != -1): #Se o artista ja esta na lista
            topArtistas[indArtista] = (artistaIterado, topArtistas[indArtista][1] + top[i]['Pontos'])
        else: #cria um novo artista
            topArtistas.append((artistaIterado, top[i]['Pontos'])) #da um append com a tupla 'nome artista' + 'pontos'
            
        #Inserindo nas arvores TRIE
        addString(trieMusicas, str(top[i]['Titulo']), indiceIniMusic)
        addString(trieArtistas, artistaIterado, indiceIniArtist)
        indiceIniMusic += 1
        indiceIniArtist += 1
    
    #Concatena o dicionario no arquivo teste.bin
    filename = 'database.bin'
    topBytes = []
    

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            topBytes = pickle.load(openfile) #carrega o que ja esta salvo no arquivo

    newData = top #Aqui vem a nova data que deve ser concatenada
        
    topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data

    with (open(filename, 'wb+')) as openfile: #abre o arquivo no modo de leitura binaria 'wb' como openfile
        pickle.dump(topBytes, openfile) 
        
    #-----------------------------------------------------------------------------    
    #Cria o arquivo de indice para consultas    
    #-----------------------------------------------------------------------------
    filename = 'indices.bin'
    fileBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            fileBytes = pickle.load(openfile)


    newData = {'Ano': ano, 'Min': len(topBytes) - 200, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
    fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(fileBytes, openfile)
     
    
    #-----------------------------------------------------------------------------
    #Cria e ordena o topMusicas - Reaproveitada da criação de arquivos
    #-----------------------------------------------------------------------------
    itera = len(topMusicas)
    for x in top:
        topMusicas.append((itera, x['Pontos']))
        if(itera < len(topMusicas)-1): #Vai incrementando ate chegar no final da lista
            itera += 1    

    topMusicas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    
    #Salva ranking geral de musicas
    with (open("topMusicas.bin", 'wb+')) as openfile:
        pickle.dump(topMusicas, openfile)  
        
    topArtistas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    
    #Salva ranking geral de artistas
    with (open("topArtistas.bin", 'wb+')) as openfile:
        pickle.dump(topArtistas, openfile)

#### Exemplo de como criar estruturas Dataframe

In [3]:
d = {'url': ['porra'], 'Song': ['BBBBB GURLS']}
df = pd.DataFrame(data=d)
print(type(df))
print(df.columns)
print("\n", df['Song'][0])

<class 'pandas.core.frame.DataFrame'>
Index(['url', 'Song'], dtype='object')

 BBBBB GURLS


### Teste de inserção de dados nos arquivos top

In [19]:
#Abrindo os arquivos necessários para adicionar mais consultas     
    with (open('topArtistas.bin', 'rb')) as openfile:
        topArtistas = pickle.load(openfile)
        
    with (open('topMusicas.bin', 'rb')) as openfile:
        topMusicas = pickle.load(openfile)

In [20]:
topArtistas.append()
topArtistas = sorted(topArtistas.items(), key=lambda kv: kv[1], reverse=True)

3701


### Teste inserção anos

In [220]:
with (open('topArtistas.bin', 'rb')) as openfile:
    topArtistas = pickle.load(openfile)
        
with (open('topMusicas.bin', 'rb')) as openfile:
    topMusicas = pickle.load(openfile)
        
#Abrindo os arquivos com as arvores Trie      
with (open('trieMusicas.bin', 'rb')) as openfile:
    trieMusicas = pickle.load(openfile)
        
with (open('trieArtistas.bin', 'rb')) as openfile:
    trieArtistas = pickle.load(openfile)
    
with (open('database.bin', 'rb')) as openfile:
    database = pickle.load(openfile)
        
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

In [184]:
df = pd.read_excel('2008Gurizada.xlsx')
print(df['SongID'][4])

4 MinutesMadonna Featuring Justin Timberlake & Timbaland


In [219]:
adicionaAno(df)

### Testes aleatorios aqui embaixo

In [218]:
print(len(database))
print(len(topMusicas))
print(len(topArtistas))
print(indices)
print('\n')
for x in topMusicas:
    print(x)

200
200
134
[{'Ano': 2005, 'Min': 0, 'Max': 199}]


(0, 3622)
(1, 3254)
(2, 3143)
(3, 2980)
(4, 2857)
(5, 2742)
(6, 2708)
(7, 2661)
(8, 2603)
(9, 2538)
(10, 2463)
(11, 2288)
(12, 2269)
(13, 2267)
(14, 2195)
(15, 2161)
(16, 2140)
(17, 2133)
(18, 2125)
(19, 2115)
(20, 2087)
(21, 2073)
(22, 2038)
(23, 2025)
(24, 2003)
(25, 1974)
(26, 1951)
(27, 1939)
(28, 1935)
(29, 1933)
(30, 1907)
(31, 1877)
(32, 1830)
(33, 1790)
(34, 1774)
(35, 1743)
(36, 1723)
(37, 1718)
(38, 1658)
(39, 1641)
(40, 1622)
(41, 1619)
(42, 1609)
(43, 1600)
(44, 1556)
(45, 1519)
(46, 1501)
(47, 1482)
(48, 1473)
(49, 1469)
(50, 1455)
(51, 1454)
(52, 1447)
(53, 1440)
(54, 1434)
(55, 1424)
(56, 1424)
(57, 1422)
(58, 1400)
(59, 1384)
(60, 1379)
(61, 1351)
(62, 1334)
(63, 1331)
(64, 1329)
(65, 1327)
(66, 1307)
(67, 1305)
(68, 1292)
(69, 1269)
(70, 1265)
(71, 1256)
(72, 1247)
(73, 1242)
(74, 1231)
(75, 1189)
(76, 1180)
(77, 1168)
(78, 1144)
(79, 1137)
(80, 1126)
(81, 1120)
(82, 1110)
(83, 1104)
(84, 1100)
(85, 1090)
(86, 1077)
(

In [221]:
print(len(database))
print(len(topMusicas))
print(len(topArtistas))
print(indices)
print('\n')
for x in topMusicas:
    print(x)

400
400
233
[{'Ano': 2005, 'Min': 0, 'Max': 199}, {'Ano': '2008', 'Min': 200, 'Max': 399}]


(0, 3622)
(1, 3254)
(200, 3207)
(2, 3143)
(3, 2980)
(200, 2871)
(4, 2857)
(201, 2805)
(202, 2763)
(203, 2760)
(204, 2747)
(5, 2742)
(6, 2708)
(7, 2661)
(8, 2603)
(205, 2577)
(9, 2538)
(206, 2466)
(10, 2463)
(207, 2427)
(208, 2380)
(209, 2325)
(210, 2313)
(211, 2311)
(11, 2288)
(12, 2269)
(13, 2267)
(14, 2195)
(212, 2189)
(213, 2174)
(214, 2171)
(15, 2161)
(16, 2140)
(17, 2133)
(215, 2126)
(18, 2125)
(19, 2115)
(216, 2093)
(20, 2087)
(217, 2075)
(21, 2073)
(218, 2063)
(22, 2038)
(23, 2025)
(219, 2016)
(24, 2003)
(220, 1991)
(221, 1982)
(25, 1974)
(222, 1973)
(26, 1951)
(27, 1939)
(223, 1936)
(28, 1935)
(29, 1933)
(30, 1907)
(31, 1877)
(32, 1830)
(224, 1827)
(33, 1790)
(34, 1774)
(225, 1767)
(226, 1744)
(35, 1743)
(227, 1728)
(36, 1723)
(228, 1723)
(37, 1718)
(229, 1715)
(230, 1712)
(231, 1711)
(232, 1704)
(233, 1690)
(234, 1685)
(235, 1678)
(236, 1663)
(38, 1658)
(237, 1655)
(39, 1641)
(238, 163

In [154]:
for x in topArtistas:
    print(x)

('Rihanna', 10094)
('Kelly Clarkson', 9894)
('Mariah Carey', 9833)
('Chris Brown', 8470)
('50 Cent', 7898)
('Lil Wayne', 7443)
('Alicia Keys', 6410)
('Gwen Stefani', 6396)
('Green Day', 6367)
('Kanye West', 6214)
('Taylor Swift', 6162)
('The Game', 5684)
('T.I.', 5299)
('The Pussycat Dolls', 5125)
('Ciara', 5103)
('Flo Rida', 5093)
('The Black Eyed Peas', 5027)
('Leona Lewis', 4892)
('Usher', 4555)
('Natasha Bedingfield', 4460)
('Coldplay', 4282)
("Destiny's Child", 4218)
('Lifehouse', 4116)
('Ne-Yo', 3987)
('Mario', 3755)
('Brad Paisley', 3730)
('Carrie Underwood', 3728)
('3 Doors Down', 3699)
('Maroon 5', 3675)
('Katy Perry', 3583)
('Bow Wow', 3533)
('Gavin DeGraw', 3489)
('T-Pain', 3480)
('Sugarland', 3445)
('Akon', 3414)
('Ludacris', 3406)
('The Killers', 3361)
('Snoop Dogg', 3344)
('Frankie J', 3318)
('Jesse McCartney', 3263)
('Colbie Caillat', 3221)
('Rob Thomas', 3134)
('Fall Out Boy', 3128)
('Rascal Flatts', 3075)
('Ying Yang Twins', 3010)
('Weezer', 2980)
('Keith Urban', 2980)

In [42]:
teste = [1,2,3,4,5]

print(teste[4])
teste.append(('herro','anothaone'))

print(teste[5])
teste.append('bais um')
print(teste)

5
('herro', 'anothaone')
[1, 2, 3, 4, 5, ('herro', 'anothaone'), 'bais um']


In [43]:
teste[5] = ('mudou', 'a tupla')
print(teste)

[1, 2, 3, 4, 5, ('mudou', 'a tupla'), 'bais um']


In [43]:
print(df.columns)
print(df['WeekID'][0])
var = df['WeekID'][0].split('-')
var = var[0]
print(var)

Index(['url', 'WeekID', 'Week Position', 'Song', 'Performer', 'SongID',
       'Instance', 'Previous Week Position', 'Peak Position',
       'Weeks on Chart'],
      dtype='object')
2008-01-05
2008
